In [ ]:
import http.server
import socketserver
import os
import hashlib
import time
from email.utils import formatdate, parsedate_to_datetime

PORT = 8081
FILENAME = "index.html"

class CachingHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path != f"/{FILENAME}":
            self.send_error(404, "File Not Found")
            return

        # Read file content
        with open(FILENAME, "rb") as f:
            content = f.read()

        # Generate ETag (MD5 of file content)
        etag = hashlib.md5(content).hexdigest()

        # Last-Modified time
        last_mod_time = os.path.getmtime(FILENAME)
        last_modified = formatdate(last_mod_time, usegmt=True)

        # Check client headers
        if_none_match = self.headers.get("If-None-Match")
        if_modified_since = self.headers.get("If-Modified-Since")

        # Validate caching
        if if_none_match == etag or (
            if_modified_since and
            parsedate_to_datetime(if_modified_since).timestamp() >= last_mod_time
        ):
            self.send_response(304)
            self.end_headers()
            return

        # Otherwise send fresh content
        self.send_response(200)
        self.send_header("Content-Type", "text/html")
        self.send_header("ETag", etag)
        self.send_header("Last-Modified", last_modified)
        self.end_headers()
        self.wfile.write(content)

with socketserver.TCPServer(("", PORT), CachingHandler) as httpd:
    print(f"Serving on port {PORT}...")
    httpd.serve_forever()


Serving on port 8081...


127.0.0.1 - - [12/Sep/2025 02:05:30] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2025 02:05:31] code 404, message File Not Found
127.0.0.1 - - [12/Sep/2025 02:05:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Sep/2025 02:05:43] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2025 02:06:07] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2025 02:06:39] "GET /index.html HTTP/1.1" 200 -
